# **This notebook will take you through the steps needed to extract the embeddings from Siamese netork**

### To know about Siamese network, refer ot this [video lecture](https://www.youtube.com/watch?v=6jfw8MuKwpI) by Andrew NG on siamese Network. 
#### Basically the idea is to ontain the embeddings for a given 2 images and compare them by l1 distance (In general as mentioned in Siamese paper, you can research on other ways of comparision like cosine similarity,euclidian distance etc.,)

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import *

## So, I am using a pretrained model ie., VGG Facenet to obtain embeddings. To know more details on how to use VGG FaceNet, Refer ot this [blog post](https://sefiks.com/2018/08/06/deep-face-recognition-with-keras/#:~:text=VGG%2DFace%20is%20deeper%20than,layer%20structre%20as%20shown%20below.). I also put the model with pretrained weights in my [Faces Dataset](https://www.kaggle.com/shanmukh05/vggface-using-tripletloss)

### Input shape will be: (BATCH_SZIE,HEIGHT,WIDTH,CHANNELS,2). I am taking l1_distance of both the embeddings and adding sigmoid layer at the top of it. 
### Finally I am taking both embed_model and siamese_model as output.

In [2]:
def create_model():
    
    inputs = tf.keras.Input(shape = (224,224,3,2,))
    
    input_a = inputs[:,:,:,:,0]
    input_b = inputs[:,:,:,:,1]
    
    pretrained_model = tf.keras.models.load_model("../input/vggface-using-tripletloss/vgg_face_model.h5")
    
    embed_model = tf.keras.Model(pretrained_model.layers[0].input,pretrained_model.layers[-2].output)
    
    embed_a = embed_model(input_a)
    embed_b = embed_model(input_b)
    
    l1_layer = Lambda(lambda tensors:tf.math.abs(tensors[0] - tensors[1]),name = "l1_layer")
    l1_distance = l1_layer([embed_a,embed_b])
    
    outputs = Dense(1,activation="sigmoid")(l1_distance)
    
    siamese_model = tf.keras.Model(inputs,outputs)
    
    return embed_model,siamese_model

embed,siamese = create_model()

### If you run the below cell, you will see that the 3rd layer from top is what we required finally {name: "model (Functional) "} (where we found embeddings of 2 images ie., **all the VGG FaceNet model is present in 3rd layer itself**. It's the key takeaway from thsi notebook where many will make mistake.)

In [3]:
siamese.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3, 0                                            
__________________________________________________________________________________________________
tf.__operators__.getitem (Slici (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem_1 (Sli (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
model (Functional)              (None, 2622)         145002878   tf.__operators__.getitem[0][0]   
                                                                 tf.__operators__.getitem_1[

In [4]:
embed.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_input (InputL [(None, 224, 224, 3)]     0         
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 226, 226, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 226, 226, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 114, 114, 64)      0     

### Getting the model as expalined above.

In [5]:
model = tf.keras.Model(siamese.layers[-3].input,siamese.layers[-3].output)
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_input (InputL [(None, 224, 224, 3)]     0         
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 226, 226, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 226, 226, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 114, 114, 64)      0   

### Copying the weights from siamese to model we obtained above

In [6]:
def copyModel2Model(model_source,model_target,certain_layer="a"):
    i=0        
    for tar,src in zip(model_target.layers,model_source.layers):
        if tar.name==certain_layer:
            break
        if i%2 !=0:
            print(model_source.layers[i].name,i)
            try:
                weights=src.get_weights()
                tar.set_weights(weights)
            except:
                i+=1
                continue
    i+=1  
    print("model source was copied into model target")
    return model_target 

embed_model = copyModel2Model(siamese,embed)
embed_model.summary()

model source was copied into model target
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_input (InputL [(None, 224, 224, 3)]     0         
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 226, 226, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 226, 226, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
zero_padding2d_2 (Z

# In this way one can obtain embeddings of siamese network.If this helped you,give it a thumbs up 🥰. Happy coding.